In [1]:
### Imports
import pandas as pd
import numpy as np
import json

In [2]:
# Step 1: Load the JSON Data
with open("sessions.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Step 2: Extract Relevant Information
callouts_data = []

for session in data:
    for section in session["sections"]:
        for callout in section["callouts"]:
            if "caller" in callout:  # Ensure there's a caller identified
                callouts_data.append(
                    {
                        "date": session["date"],
                        "period": session["period"],
                        "session": session["sessionNumber"],
                        "speaker_id": section["speaker"],
                        "caller_id": callout.get("caller"),
                    }
                )

# Step 3: Create the DataFrame
df_callouts = pd.DataFrame(callouts_data)

# Display the first few rows of the DataFrame to verify
print(df_callouts)

                       date period  session speaker_id caller_id
0       2023-09-20T00:00:00  XXVII      230      14795     52687
1       2023-09-20T00:00:00  XXVII      230      14795     35514
2       2023-09-20T00:00:00  XXVII      230      14795     35514
3       2023-09-20T00:00:00  XXVII      230       6486     35520
4       2023-09-20T00:00:00  XXVII      230       6486     35520
...                     ...    ...      ...        ...       ...
136287  2002-12-20T00:00:00   XXII        1      13032      2768
136288  2002-12-20T00:00:00   XXII        1      13032      8243
136289  2002-12-20T00:00:00   XXII        1       8240      2964
136290  2002-12-20T00:00:00   XXII        1       8182      2964
136291  2002-12-20T00:00:00   XXII        1        946      2821

[136292 rows x 5 columns]


In [3]:
# Load the persons data ('persons.json')
with open("persons.json", "r", encoding="utf-8") as file:
    persons_data = json.load(file)

# Create the lookup tables
persons_lookup = {
    person["id"]: {
        "name": person["givenName"] + " " + person["familyName"],
        "parties": person["parties"],
    }
    for person in persons_data
}

# Update the DataFrame with caller_name, speaker_name, caller_parties, and speaker_parties
df_callouts["caller_name"] = df_callouts["caller_id"].map(
    lambda x: persons_lookup.get(x, {}).get("name", "Unknown")
)
df_callouts["speaker_name"] = df_callouts["speaker_id"].map(
    lambda x: persons_lookup.get(x, {}).get("name", "Unknown")
)
df_callouts["caller_parties"] = df_callouts["caller_id"].map(
    lambda x: ", ".join(persons_lookup.get(x, {}).get("parties", []))
)
df_callouts["speaker_parties"] = df_callouts["speaker_id"].map(
    lambda x: ", ".join(persons_lookup.get(x, {}).get("parties", []))
)

# Display the updated DataFrame
print(df_callouts)

                       date period  session speaker_id caller_id  \
0       2023-09-20T00:00:00  XXVII      230      14795     52687   
1       2023-09-20T00:00:00  XXVII      230      14795     35514   
2       2023-09-20T00:00:00  XXVII      230      14795     35514   
3       2023-09-20T00:00:00  XXVII      230       6486     35520   
4       2023-09-20T00:00:00  XXVII      230       6486     35520   
...                     ...    ...      ...        ...       ...   
136287  2002-12-20T00:00:00   XXII        1      13032      2768   
136288  2002-12-20T00:00:00   XXII        1      13032      8243   
136289  2002-12-20T00:00:00   XXII        1       8240      2964   
136290  2002-12-20T00:00:00   XXII        1       8182      2964   
136291  2002-12-20T00:00:00   XXII        1        946      2821   

                 caller_name              speaker_name caller_parties  \
0               Alois Stöger           August Wöginger            SPÖ   
1            Wolfgang Zanger         

In [80]:
df_callouts.to_csv(
    "df_callouts_total.tsv", sep="\t", index=False
)  # save to TSV because initial parsing takes long

In [4]:
df_callouts_XXVII = df_callouts[(df_callouts["period"] == "XXVII")]
# find people with multiple party asociations and manually correct for most current afiliation
multiple_parties = (df_callouts["caller_parties"].str.contains(",", na=False)) | (
    df_callouts["speaker_parties"].str.contains(",", na=False)
)

party_corrections = {"2345": "GRÜNE", "51577": "FPÖ", "2867": "FPÖ"}

for id_str, correct_party in party_corrections.items():
    # Check for matches in 'caller_id' and update 'caller_parties'
    match_caller = df_callouts_XXVII["caller_id"] == id_str
    df_callouts_XXVII.loc[match_caller, "caller_parties"] = correct_party

    # Check for matches in 'speaker_id' and update 'speaker_parties'
    match_speaker = df_callouts_XXVII["speaker_id"] == id_str
    df_callouts_XXVII.loc[match_speaker, "speaker_parties"] = correct_party

# df_callouts_XXVII.replace("", np.nan, inplace=True)
df_callouts_XXVII

,date,period,session,speaker_id,caller_id,caller_name,speaker_name,caller_parties,speaker_parties
0,2023-09-20T00:00:00,XXVII,230,14795,52687,Alois Stöger,August Wöginger,SPÖ,ÖVP
1,2023-09-20T00:00:00,XXVII,230,14795,35514,Wolfgang Zanger,August Wöginger,FPÖ,ÖVP
2,2023-09-20T00:00:00,XXVII,230,14795,35514,Wolfgang Zanger,August Wöginger,FPÖ,ÖVP
3,2023-09-20T00:00:00,XXVII,230,6486,35520,Herbert Kickl,Susanne Raab,FPÖ,ÖVP
4,2023-09-20T00:00:00,XXVII,230,6486,35520,Herbert Kickl,Susanne Raab,FPÖ,ÖVP
...,...,...,...,...,...,...,...,...,...
23109,2019-10-23T00:00:00,XXVII,1,35468,83122,Beate Meinl-Reisinger,Dagmar Belakowitsch,NEOS,FPÖ
23110,2019-10-23T00:00:00,XXVII,1,35468,83122,Beate Meinl-Reisinger,Dagmar Belakowitsch,NEOS,FPÖ
23111,2019-10-23T00:00:00,XXVII,1,87146,83122,Beate Meinl-Reisinger,Ewa Ernst-Dziedzic,NEOS,GRÜNE
23112,2019-10-23T00:00:00,XXVII,1,87146,35520,Herbert Kickl,Ewa Ernst-Dziedzic,FPÖ,GRÜNE


In [6]:
# Add a prefix to caller_id and speaker_id
df_callouts_XXVII["caller_id"] = "c" + df_callouts_XXVII["caller_id"].astype(str)
df_callouts_XXVII["speaker_id"] = "s" + df_callouts_XXVII["speaker_id"].astype(str)

df_callouts_XXVII_grouped = (
    df_callouts_XXVII.groupby(
        [
            "caller_id",
            "speaker_id",
            "caller_name",
            "speaker_name",
            "caller_parties",
            "speaker_parties",
        ]
    )
    .size()
    .reset_index(name="counts")
)

# Assuming df_callouts_XXVII_grouped is your starting DataFrame
# First, calculate the total counts for each caller_id
total_counts_per_caller = (
    df_callouts_XXVII_grouped.groupby("caller_id")["counts"]
    .sum()
    .reset_index(name="counts_total")
)

# Then, merge this total counts back into your original (or grouped) DataFrame
df_callouts_XXVII_grouped_with_totals = pd.merge(
    df_callouts_XXVII_grouped, total_counts_per_caller, on="caller_id", how="left"
)

# Sorting the DataFrame based on counts_total (you might also want to keep the original sorting as a secondary criterion)
df_sorted_by_total_counts = df_callouts_XXVII_grouped_with_totals.sort_values(
    by=["counts_total", "counts"], ascending=False
)


df_sorted_by_total_counts.to_csv(
    "df_callouts_XXVII.tsv", sep="\t", index=False
)  # save data as final .tsv

df_sorted_by_total_counts

/tmp/ipykernel_46927/1355460316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callouts_XXVII["caller_id"] = "c" + df_callouts_XXVII["caller_id"].astype(str)
/tmp/ipykernel_46927/1355460316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_callouts_XXVII["speaker_id"] = "s" + df_callouts_XXVII["speaker_id"].astype(str)


,caller_id,speaker_id,caller_name,speaker_name,caller_parties,speaker_parties,counts,counts_total
1671,cc35468,ss2136,Dagmar Belakowitsch,Karl Nehammer,FPÖ,ÖVP,146,2497
1737,cc35468,ss5676,Dagmar Belakowitsch,Ralph Schallmeiner,FPÖ,GRÜNE,112,2497
1770,cc35468,ss83101,Dagmar Belakowitsch,Sigrid Maurer,FPÖ,GRÜNE,95,2497
1638,cc35468,ss14795,Dagmar Belakowitsch,August Wöginger,FPÖ,ÖVP,85,2497
1759,cc35468,ss65321,Dagmar Belakowitsch,Sebastian Kurz,FPÖ,ÖVP,78,2497
...,...,...,...,...,...,...,...,...
3597,cc5686,ss2136,Johannes Margreiter,Karl Nehammer,NEOS,ÖVP,1,1
3712,cc6506,ss6486,Romana Deckenbacher,Susanne Raab,ÖVP,ÖVP,1,1
3770,cc65321,ss78586,Sebastian Kurz,Christian Hafenecker,ÖVP,FPÖ,1,1
3808,cc7106,ss145,Werner Saxinger,Doris Bures,ÖVP,SPÖ,1,1


In [7]:
# Extract unique caller and speaker information including party
unique_callers = (
    df_callouts_XXVII[["caller_id", "caller_name", "caller_parties"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
unique_speakers = (
    df_callouts_XXVII[["speaker_id", "speaker_name", "speaker_parties"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Format the IDs, names, and parties, and combine into a single list of nodes
nodes = [
    {
        "id": f"c{row['caller_id']}",
        "name": row["caller_name"],
        "party": row["caller_parties"],
    }
    for index, row in unique_callers.iterrows()
]
nodes += [
    {
        "id": f"s{row['speaker_id']}",
        "name": row["speaker_name"],
        "party": row["speaker_parties"],
    }
    for index, row in unique_speakers.iterrows()
]

# Extract and format the interruptions
interruptions = [
    {
        "caller": f"c{row['caller_id']}",
        "speaker": f"s{row['speaker_id']}",
        "session": row["session"],
        "date": row["date"]
    }
    for index, row in df_callouts_XXVII.iterrows()
]


# Combine into a single structure
data_structure = {"nodes": nodes, "interruptions": interruptions}

# Save to JSON file
with open("callouts_structure.json", "w", encoding="utf-8") as file:
    json.dump(data_structure, file, ensure_ascii=False, indent=4)